In [1]:
from pymilvus import utility, connections
connections.connect(
  alias="default",
  user='username',
  password='password',
  host='localhost',
  port='19530'
)

In [14]:
from pymilvus import CollectionSchema, FieldSchema, DataType, Collection

auto_id = FieldSchema(
    name="id", 
    dtype=DataType.INT64, 
    descrition="primary field", 
    is_primary=True, 
    auto_id=True
)

date = FieldSchema(
  name="date",
  dtype=DataType.VARCHAR,
  max_length=200
)
open = FieldSchema(
  name="open",
  dtype=DataType.INT64
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
)
high = FieldSchema(
  name="high",
  dtype=DataType.INT64
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
)
low = FieldSchema(
  name="low",
  dtype=DataType.INT64
)
close = FieldSchema(
  name="close",
  dtype=DataType.INT64
)

volume = FieldSchema(
  name="volume",
  dtype=DataType.INT64
)

stock_intro = FieldSchema(
  name="stock_intro",
  dtype=DataType.FLOAT_VECTOR,
  dim=2
)

schema = CollectionSchema(
  fields = [auto_id, date, open, high, low, close, volume, stock_intro],
  description="stock prices",
  enable_dynamic_field=True
)
collection_name = "stocks"

collection = Collection(collection_name, schema)

In [15]:
tasks = utility.do_bulk_insert(
    collection_name=collection_name,
    is_row_based=True,
    files=["TSLA_daily.csv"]
)

In [35]:
index_params = {
  "metric_type":"L2",
  "index_type":"IVF_FLAT",
  "params":{"nlist":1024}
}

In [36]:
from pymilvus import Collection, utility
# Get an existing collection.
collection = Collection(collection_name)      
collection.create_index(
  field_name="stock_intro", 
  index_params=index_params
)

utility.index_building_progress(collection_name)
# Output: {'total_rows': 0, 'indexed_rows': 0}

{'total_rows': 0, 'indexed_rows': 0, 'pending_index_rows': 0}

In [30]:
price_limit=200

In [44]:
f'low > {price_limit}'

'low > 200'

In [41]:
collection.load()

expr_ = f'low > {price_limit}'
res = collection.query(
  expr = expr_.format(price_limit),
  output_fields = ["stock_name", "volume"],
)

In [43]:
res

[]